# ✅ Asynkron Solana-pipeline med støtte for transaction versions

📦 Bruker `aiohttp` for å sende rå RPC-kall til Solana og håndtere `maxSupportedTransactionVersion`

🔗 Referanse: [ChatGPT-samtale](https://chatgpt.com/share/e/688d1ed5-04f0-800f-a579-80b2a559e77c)  
👤 GitHub-bruker: `sac-marius`  
🛠️ Bruker `asyncio` + `aiohttp` for å hente blokker uten feil knyttet til transaksjonsversjoner

---


In [ ]:
# Installer nødvendige pakker
!pip install aiohttp pandas pyarrow tqdm nest_asyncio


In [ ]:
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd
from tqdm.notebook import tqdm
import time

nest_asyncio.apply()


In [ ]:
RPC_URL = "https://api.mainnet-beta.solana.com"

HEADERS = {
    "Content-Type": "application/json"
}

async def fetch_block(session, slot):
    payload = {
        "jsonrpc": "2.0",
        "id": slot,
        "method": "getBlock",
        "params": [
            slot,
            {
                "maxSupportedTransactionVersion": 0
            }
        ]
    }

    try:
        async with session.post(RPC_URL, headers=HEADERS, data=json.dumps(payload)) as resp:
            res = await resp.json()
            result = res.get("result", None)
            if not result:
                return []

            block_time = result.get("blockTime")
            txs = result.get("transactions", [])
            parsed = []
            for tx in txs:
                message = tx['transaction']['message']
                meta = tx['meta']
                if not meta or not meta.get('postBalances'):
                    continue

                accounts = message['accountKeys']
                from_addr = accounts[0]
                to_addr = accounts[1] if len(accounts) > 1 else None

                lamports = (
                    meta['postBalances'][1] - meta['preBalances'][1]
                    if len(meta['postBalances']) > 1 else 0
                )

                parsed.append({
                    'slot': slot,
                    'timestamp': block_time,
                    'tx_signature': tx['transaction']['signatures'][0],
                    'from_address': from_addr,
                    'to_address': to_addr,
                    'lamports': lamports
                })
            return parsed
    except Exception as e:
        print(f"⚠️ Feil på slot {slot}: {e}")
        return []


In [ ]:
async def main():
    # Manuelt hent siste slot fra RPC hvis ønskelig
    latest_slot = 357232500  # Sett manuelt om ønskelig
    n = 1000
    slots = list(range(latest_slot - n, latest_slot))

    all_results = []
    start = time.time()

    connector = aiohttp.TCPConnector(limit=10)  # 10 samtidige kall
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [fetch_block(session, slot) for slot in slots]
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Henter blokker"):
            result = await f
            all_results.extend(result)

    df = pd.DataFrame(all_results)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.to_parquet("solana_blocks_async.parquet", index=False, compression='snappy')

    print("✅ Lagret som 'solana_blocks_async.parquet'")
    print(f"⏱️ Total tid: {time.time() - start:.2f} sekunder")

await main()
